# AMAR, HMM mode construction
In our first pass on jump data on the Galaxy Exercise Device, we develop relatively rudimentary analyses on the time series.

## Preliminaries

In [ ]:
print("heloo")

In [ ]:
from pandas import Series, DataFrame
import pandas
import matplotlib
from sklearn.preprocessing import MinMaxScaler
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [ ]:
__DEBUG2__ = False

series = pandas.read_csv('data/jump180905.csv', header=0, index_col=0)
print(type(series))

### Moving Windows
We apply moving averages and variance using various moving windows.

### Scaling
For visualization purposes, we apply a basic min-max scaler.

In [ ]:
# for row in series:
# 	print(row.iloc(2))
# 	break

values = series.values
print("Values[0]:", values[0])
print("Values shape:", values.shape)
# print(values[:,0])

for i in range(2):
	continue
	data = values[:,i]
	scaler = MinMaxScaler(feature_range=(0, 1))
	
	scaler = scaler.fit(data.reshape(-1, 1))
	print('Min: %f, Max: %f' % (scaler.data_min_[0], scaler.data_max_[0]))
	normalized = scaler.transform(values)
	for i in range(5):
		print(normalized[i])

data = values
scaler = MinMaxScaler(feature_range=(-1, 1))

scaler = scaler.fit(data)
print('Min: %f, Max: %f' % (scaler.data_min_[0], scaler.data_max_[0]))
print('Min: %f, Max: %f' % (scaler.data_min_[1], scaler.data_max_[1]))


normalized = scaler.transform(values)
if __DEBUG2__:
	for i in range(5):
		print(normalized[i])

print(series.shape)
print(normalized.shape)

# series.values = normalized

df = DataFrame(
	data = normalized, 
	index = series.index, 
	columns = ["load", "center_of_pressure"])

df['var'] = df['load'].rolling(window=100).var()
df['avload'] = df['load'].rolling(window=1500, win_type="triang").mean()


# for i in range(5):
# 	print(inversed[i])

df.plot()
plt.show()

## AMAR
We fit an AMAR model to our data. The model has the form 
$$x_n=\sum^m_{k=1}A_kx_{n-k}$$. 
The vector $x_n$ represents the state at time $t$, the weights $A_k$ are matrices. The optimization objective is to find $A_k$ that minimize the difference between model predicted $x_n$ and the training data value. Doing so is a simple linear quadratic regression problem. We proceed thus: